In [1]:
import joblib 
import pandas as pd

# Hides all warnings
import warnings
warnings.filterwarnings("ignore")
scaler_filename = "../models/scaler.save"
scaler = joblib.load(scaler_filename)

col_types = {'Gender':"str",
             'Married':"str",
             'Dependents':"str",
             'Education':"str",
             'Self_Employed':"str",
             'ApplicantIncome':"float",
             'CoapplicantIncome':"float",
             'LoanAmount':"float",
             'Loan_Amount_Term':"float",
             'Credit_History':"bool",
             'Property_Area':"str",
             'Loan_Status':"str"}

def transform(data):
    df = pd.DataFrame.from_dict(data, orient='index').transpose()
    
    # Drop Unecessary Variables
    if 'Loan_ID' in df.columns:
        df = df.drop(['Loan_ID'], axis=1)
    
    # Set Variable types
    df = df.astype(dtype = {key: value for key, value in col_types.items() if key != 'Loan_Status'})
    df = df.replace("nan", None)
    
    # One-hot Encoding of Categorical Variables
    df['Gender'] = df['Gender'].replace({'Female': False, 'Male': True})
    df['Married'] = df['Married'].replace({'No': False, 'Yes': True})
    df['Education'] = df['Education'].replace({'Not Graduate': False, 'Graduate': True})
    df['Self_Employed'] = df['Self_Employed'].replace({'No': False, 'Yes': True})
    df['Dependents_0'] = df['Dependents'].apply(lambda x: True if x == '0' else False)
    df['Dependents_1'] = df['Dependents'].apply(lambda x: True if x == '1' else False)
    df['Dependents_2'] = df['Dependents'].apply(lambda x: True if x == '2' else False)
    df['Dependents_3+'] = df['Dependents'].apply(lambda x: True if x == '3+' else False)
    df = df.drop('Dependents', axis=1)
    df['Property_Area_Rural'] = df['Property_Area'].apply(lambda x: True if x == 'Rural' else False)
    df['Property_Area_Semiurban'] = df['Property_Area'].apply(lambda x: True if x == 'Semiurban' else False)
    df['Property_Area_Urban'] = df['Property_Area'].apply(lambda x: True if x == 'Urban' else False)
    df = df.drop('Property_Area', axis=1)
    
    # Data Normalization Over Data Set
    df = scaler.transform(df)

    return df.astype(np.float32)

In [3]:
import requests
import json
import numpy as np

# Define the URL of the Flask service
url = "https://loan-loan-prediction-mo.apps.prod.rhoai.rh-aiservices-bu.com/v2/models/loan/infer"


# Define the JSON payload
payload = {
    "Loan_ID": "LP001002",
    "Gender": "Male",
    "Married": "No",
    "Dependents": "0",
    "Education": "Graduate",
    "Self_Employed": "No",
    "ApplicantIncome": 5849,
    "CoapplicantIncome": 0.0,
    "LoanAmount": 120,
    "Loan_Amount_Term": 360.0,
    "Credit_History": 1.0,
    "Property_Area": "Urban",
}

try:
    json_payload = json.dumps({"input":transform(payload).tolist()})
    print("JSON Payload:", json_payload)
except (TypeError, ValueError) as e:
    print("Error serializing to JSON:", e)
# Send the POST request
response = requests.post(url,json_payload ,verify=False)

# Print the response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())

JSON Payload: {"input": [[1.0, 0.0, 1.0, 0.0, 48.99017333984375, 0.0, 7.978983402252197, 0.7435897588729858, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]}
Status Code: 400
Response JSON: {'error': 'Invalid JSON structure, missing inputs'}
